<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.64
    Uninstalling yfinance-0.2.64:
      Successfully uninstalled yfinance-0.2.64
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-02 12:16:54
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  6.47 K
Current:  1.36 C
-------------------
Today PnL: -12.57 K (-0.09%)
Current PnL: -13.68 L (-9.53%)
CY Booked + Current PnL: -6.94 L (-4.84%)
-------------------
Total profit:  4.55 L
Total loss:  -18.23 L
-------------------
Total Booked + Current PnL: 20.04 L (17.19%)
Total Booked PnL: 33.72 L (28.93%)
Curr Year Booked PnL: 6.74 L (4.95%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.08 C
Est FTT PnL: 72.11 L (52.94%)
Deployed:  1.17 C
Current:  1.36 C
CAGR/XIRR %: 10.63%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
88,UNIONBANK,123.87,163.00,1.69,M-LC,45.83,176235.0,35395.0,9094.0,0.23,25.13,5.16,31.59,89.0,3.89,1.29,53.09,XY24,NTT,BANKS
3,ADANIPORTS,1103.69,1583.00,2.40,M-LC,26.04,102801.0,24439.0,9591.0,0.06,31.19,9.33,43.43,77.0,2.55,0.76,38.16,XY24,NTT,MISC
87,UJJIVANSFB,52.77,53.00,52.86,M-SC,88.54,132975.0,-9504.0,10119.0,0.06,-6.67,7.61,0.44,243.0,-0.94,0.98,56.95,OX40N,NTT,BANKS
39,HINDALCO,651.95,761.55,-19.08,H-LC,91.67,111088.0,6776.0,10764.0,0.21,6.50,9.69,16.81,27.0,0.63,0.82,23.58,X5K,ATH,METALS
5,APOLLOHOSP,6904.43,8285.00,-17.40,H-LC,81.25,112500.0,8934.0,11779.0,0.05,8.63,10.47,20.00,30.0,0.76,0.83,23.91,X40N,BTT,HEALTHCARE


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.2,-28.73,L-SC,100.0,142978.0,1136.0,55433.0,-1.05,0.8,38.77,39.88,259.0,0.02,1.05,44.9,XY24,NTT,POWER


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,DIGITIDE,188.38,237.72,NaN,NaN,2.08,45024.0,3769.0,7037.0,6.00,9.14,15.63,26.19,1.0,0.54,0.33,6.00,XY24,ATH,IT
16,BLUSPRING,226.45,84.55,NaN,NaN,6.25,16972.0,-32621.0,1544.0,5.45,-65.78,9.10,-62.66,NaN,-21.13,0.12,13.82,XY24,ATH,MISC
63,QUESS,296.83,986.00,-30.58,M-SC,36.98,68700.0,3694.0,147231.0,4.51,5.68,214.31,232.18,191.0,0.03,0.50,5.20,XY24,NTT,MISC
60,NATIONALUM,189.63,247.44,-48.64,H-MC,77.08,102931.0,2427.0,28213.0,2.07,2.41,27.41,30.49,97.0,0.09,0.76,35.60,MH,ATH,METALS
68,SAIL,130.64,228.00,94.40,M-MC,58.33,233400.0,8438.0,159225.0,1.65,3.75,68.22,74.53,185.0,0.05,1.71,35.51,XY24,BTT,STEEL


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
24,COFFEEDAY,59.14,80.00,-53.72,L-SC,96.88,68678.0,-44871.0,84920.0,-2.21,-39.52,123.65,35.27,261.0,-0.53,0.50,68.09,XR,NTT,HOTELS
69,SAMMAANCAP,170.35,326.00,-171.59,M-SC,89.58,82416.0,-19794.0,113182.0,-2.17,-19.37,137.33,91.37,206.0,-0.17,0.61,33.79,XY25,NTT,FINANCE
54,KPIGREEN,497.21,732.16,27.50,H-SC,65.62,130536.0,5239.0,53964.0,-2.05,4.18,41.34,47.25,139.0,0.10,0.96,63.31,MH,ATH,POWER
47,INDUSINDBK,1354.50,1800.00,81.30,L-MC,69.79,49157.0,-28050.0,53443.0,-1.90,-36.33,108.72,32.89,253.0,-0.52,0.36,35.37,XR,NTT,BANKS
66,REPCOHOME,515.07,880.00,-53.06,H-SC,76.04,172274.0,-32724.0,177959.0,-1.66,-15.96,103.30,70.85,133.0,-0.18,1.27,39.07,XY24,NTT,FINANCE


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,UJJIVANSFB,52.77,53.0,52.86,M-SC,88.54,132975.0,-9504.0,10119.0,0.06,-6.67,7.61,0.44,243.0,-0.94,0.98,56.95,OX40N,NTT,BANKS
92,VIPIND,488.80,489.0,-213.31,H-SC,97.92,81724.0,-13592.0,13632.0,-1.53,-14.26,16.68,0.04,154.0,-1.00,0.60,61.19,OX40N,NTT,MISC
79,SYMPHONY,1306.42,1306.0,-32.03,M-SC,5.21,144729.0,-26412.0,26355.0,-1.22,-15.43,18.21,-0.03,189.0,-1.00,1.06,4.02,OX40N,NTT,DURABLES
46,INDIGOPNTS,1407.73,1408.0,123.77,H-SC,87.50,146766.0,-27793.0,27827.0,0.26,-15.92,18.96,0.02,127.0,-1.00,1.08,27.27,OX40N,NTT,PAINTS
81,TATAELXSI,7332.28,7332.0,-18.32,X-MC,15.62,85827.0,-16825.0,16822.0,-1.26,-16.39,19.60,-0.00,58.0,-1.00,0.63,30.47,OX40N,NTT,IT


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,DIGITIDE,188.38,237.72,NaN,NaN,2.08,45024.0,3769.0,7037.0,6.00,9.14,15.63,26.19,1.0,0.54,0.33,6.00,XY24,ATH,IT
83,TCS,3794.03,4998.00,-17.74,X-LC,43.75,268242.0,-27692.0,121594.0,0.27,-9.36,45.33,31.73,1.0,-0.23,1.97,7.37,X40,BTT,IT
48,INFY,1461.46,2275.00,-9.91,X-LC,59.38,288691.0,27090.0,118537.0,0.37,10.36,41.06,55.67,2.0,0.23,2.12,17.09,X40,BTT,IT
40,HINDUNILVR,2413.81,2723.00,-17.68,X-LC,21.88,257600.0,-12747.0,47373.0,0.20,-4.71,18.39,12.81,6.0,-0.27,1.89,7.73,XY25,NTT,FMCG
61,NESTLEIND,2268.60,2755.00,-11.84,X-LC,28.12,279618.0,14192.0,42726.0,-0.84,5.35,15.28,21.44,9.0,0.33,2.05,11.88,XY25,NTT,FMCG


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
88,UNIONBANK,123.87,163.0,1.69,M-LC,45.83,176235.0,35395.0,9094.0,0.23,25.13,5.16,31.59,89.0,3.89,1.29,53.09,XY24,NTT,BANKS
3,ADANIPORTS,1103.69,1583.0,2.40,M-LC,26.04,102801.0,24439.0,9591.0,0.06,31.19,9.33,43.43,77.0,2.55,0.76,38.16,XY24,NTT,MISC
32,ENRIN,1377.95,2975.9,NaN,NaN,73.96,73095.0,38646.0,1301.0,-1.61,112.18,1.78,115.97,NaN,29.70,0.54,11.77,AR,ATH,ELECTRICAL


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SAIL,130.64,228.00,94.40,M-MC,58.33,233400.0,8438.0,159225.0,1.65,3.75,68.22,74.53,185.0,0.05,1.71,35.51,XY24,BTT,STEEL
54,KPIGREEN,497.21,732.16,27.50,H-SC,65.62,130536.0,5239.0,53964.0,-2.05,4.18,41.34,47.25,139.0,0.10,0.96,63.31,MH,ATH,POWER
20,CAMS,3643.00,5250.99,5.87,H-SC,52.08,119028.0,17024.0,27995.0,0.19,16.69,23.52,44.14,124.0,0.61,0.87,35.64,X40N,ATH,MISC
3,ADANIPORTS,1103.69,1583.00,2.40,M-LC,26.04,102801.0,24439.0,9591.0,0.06,31.19,9.33,43.43,77.0,2.55,0.76,38.16,XY24,NTT,MISC
88,UNIONBANK,123.87,163.00,1.69,M-LC,45.83,176235.0,35395.0,9094.0,0.23,25.13,5.16,31.59,89.0,3.89,1.29,53.09,XY24,NTT,BANKS


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,2975.90,NaN,NaN,73.96,73095.0,38646.0,1301.0,-1.61,112.18,1.78,115.97,NaN,29.70,0.54,11.77,AR,ATH,ELECTRICAL
94,WHIRLPOOL,1167.49,2270.00,-37.67,M-SC,93.75,205374.0,33753.0,128318.0,1.28,19.67,62.48,94.43,198.0,0.26,1.51,51.58,XR,NTT,DURABLES
95,WIPRO,243.46,420.00,-3.77,M-LC,78.12,164920.0,13975.0,95472.0,0.55,9.26,57.89,72.51,69.0,0.15,1.21,13.60,XR,NTT,IT
45,INDIAMART,2327.09,4871.06,-47.70,H-SC,90.62,136252.0,12916.0,121918.0,-1.04,10.47,89.48,109.32,122.0,0.11,1.00,34.61,AR,ATH,MISC
54,KPIGREEN,497.21,732.16,27.50,H-SC,65.62,130536.0,5239.0,53964.0,-2.05,4.18,41.34,47.25,139.0,0.10,0.96,63.31,MH,ATH,POWER


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
24,COFFEEDAY,59.14,80.00,-53.72,L-SC,96.88,68678.0,-44871.0,84920.0,-2.21,-39.52,123.65,35.27,261.0,-0.53,0.50,68.09,XR,NTT,HOTELS
58,MASFIN,326.60,402.39,-18.25,H-SC,64.58,92940.0,-5040.0,27780.0,0.06,-5.14,29.89,23.21,137.0,-0.18,0.68,34.80,XR,ATH,FINANCE
4,ALKYLAMINE,2347.98,4546.37,5.92,X-SC,98.96,98603.0,-2360.0,96887.0,-0.42,-2.34,98.26,93.63,64.0,-0.02,0.72,46.87,SR,ATH,CHEMICALS
3,ADANIPORTS,1103.69,1583.00,2.40,M-LC,26.04,102801.0,24439.0,9591.0,0.06,31.19,9.33,43.43,77.0,2.55,0.76,38.16,XY24,NTT,MISC
90,VALIANTORG,512.64,838.00,-350.78,H-SC,4.17,106600.0,-26686.0,111280.0,-1.00,-20.02,104.39,63.47,148.0,-0.24,0.78,78.69,XR,NTT,CHEMICALS


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.20,-28.73,L-SC,100.00,142978.0,1136.0,55433.0,-1.05,0.80,38.77,39.88,259.0,0.02,1.05,44.90,XY24,NTT,POWER
4,ALKYLAMINE,2347.98,4546.37,5.92,X-SC,98.96,98603.0,-2360.0,96887.0,-0.42,-2.34,98.26,93.63,64.0,-0.02,0.72,46.87,SR,ATH,CHEMICALS
92,VIPIND,488.80,489.00,-213.31,H-SC,97.92,81724.0,-13592.0,13632.0,-1.53,-14.26,16.68,0.04,154.0,-1.00,0.60,61.19,OX40N,NTT,MISC
24,COFFEEDAY,59.14,80.00,-53.72,L-SC,96.88,68678.0,-44871.0,84920.0,-2.21,-39.52,123.65,35.27,261.0,-0.53,0.50,68.09,XR,NTT,HOTELS
51,JIOFIN,310.58,387.00,6.78,H-LC,95.83,228900.0,11494.0,42003.0,-0.79,5.29,18.35,24.61,95.0,0.27,1.68,62.75,XY24,BTT,FINANCE


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.86
1,25,41.92
2,50,70.46


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.48
LC    34.04
MC    21.48
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.59
X40      12.24
XY25     12.01
X40N     11.80
XR       10.15
AR        8.23
OX40N     8.16
MH        1.72
X5K       1.48
X200      1.40
SR        1.21
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.67
M-SC    16.00
H-LC    14.11
X-LC    12.19
H-MC     9.50
X-MC     7.51
M-LC     6.69
X-SC     4.34
M-MC     4.11
L-SC     2.47
L-LC     1.05
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.08,-1.92,35.98
IT,12.09,-10.36,64.58
BANKS,8.02,-4.44,47.06
FINANCE,7.91,-12.06,52.62
MISC,6.65,-14.80,68.06
PAINTS,5.67,-13.13,35.42
ELECTRICAL,5.46,0.78,45.56
HEALTHCARE,4.51,-4.86,35.49
AUTO,4.32,-14.58,64.96


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2699168.0,27
XR,1075362.0,13
AR,1024983.0,10
X40,653576.0,9
X40N,556594.0,11
XY25,484337.0,8
OX40N,320554.0,11
SR,178679.0,2
MH,82177.0,2


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1972072.0,20
M-SC,1764613.0,21
H-LC,617640.0,12
H-MC,541155.0,8
X-LC,515249.0,7
X-MC,430286.0,6
M-MC,408830.0,3
X-SC,327715.0,5
M-LC,262410.0,5


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      861435.0      6
M-SC       XY24      813396.0      7
H-SC       AR        522889.0      3
M-SC       XR        333046.0      4
M-MC       XY24      302381.0      2
H-MC       XY24      300126.0      3
X-LC       X40       240131.0      2
H-SC       X40N      225200.0      4
           XR        216130.0      3
H-LC       X40N      200721.0      4
M-SC       AR        181989.0      2
           OX40N     169001.0      5
H-LC       X40       159079.0      3
M-LC       XY24      154258.0      3
X-MC       X40       152887.0      2
L-SC       XR        146798.0      2
X-SC       XR        124024.0      1
X-LC       XY25      118565.0      3
           AR        116604.0      1
M-SC       XY25      113182.0      1
M-MC       XR        106449.0      1
X-MC       XY25      105029.0      1
           XY24       97028.0      1
X-SC       SR         96887.0      1
M-LC       XR         95472.0      1
H-SC       OX40N      92454.0      3
H-MC       XY25       88803.0      1
M-SC       SR         81792.0      1
H-LC       AR         80253.0      1
M-SC       X40        72207.0      1
H-LC       X200       69066.0      1
           X5K        66518.0      2
X-SC       XY24       64527.0      1
H-MC       AR         62537.0      1
L-SC       AR         59410.0      1
X-MC       X40N       58520.0      1
L-SC       XY24       55433.0      1
H-SC       MH         53964.0      1
L-MC       XR         53443.0      1
L-LC       XY25       46078.0      1
X-SC       OX40N      42277.0      2
H-LC       XY24       42003.0      1
X-LC       X40N       39949.0      1
H-MC       X40N       32204.0      1
           X40        29272.0      1
           MH         28213.0      1
X-MC       OX40N      16822.0      1
M-LC       XY25       12680.0      1

In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 36.0 seconds
